# Chapter 11. Interfaces: From protocols to ABCs

# Interfaces and Protocols in Python Culture
- Roughly speaking, Protocols, Duck Typing, and Interfaces are the same mean in Python

# Python Digs Sequences
- if there aren't `__len__`, `__iter__`, `__contains__`, the instance of class implementing `__getitem__` can be iterated by interpreter

In [8]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [9]:
f = Foo()
f[1]

10

In [10]:
for i in f: print(i)

0
10
20


In [11]:
20 in f

True

In [12]:
15 in f

False

In [27]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

# Monkey-Patching to Implement a Protocol at Runtime

In [14]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[7, 3, 6, 8, 2, 0, 5, 1, 9, 4]

In [28]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [29]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [31]:
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='2', suit='spades'),
 Card(rank='9', suit='clubs'),
 Card(rank='7', suit='spades'),
 Card(rank='J', suit='hearts'),
 Card(rank='A', suit='clubs')]

# Duck Typing
> duck typing (i.e, ignoring an object's actural type, forcusing instead on ensuring that the object implements the method names, signatures, and semantics required for its intended use).  
<br>
by Alex Martelli   

- Programmers can use template classes (ABCs) simpliy by minimum implementing required methods. Of cause, can register or inhefit those explicitly for readability.
- It's usually Not OK to have a chain of `if/elif/elif` with `isinstance` checks.
    - use polymorphism instead.
- Outside of frameworks, duck typing is often simpler and more flexible than type checks.

In [32]:
# Duck typing to handle a string or an iterable of strings
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)

NameError: name 'field_names' is not defined

# Subclassing an ABC

In [44]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, card):
        self._cards[position] = card
    
    def __delitem__(self, position):
        del self._cards[position]
    
    def insert(self, position, value):
        self._cards.insert(position, value)

/Users/tsubasa/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """


# ABCs in the Standard Library

## collections.abc (Fig.11-3)
- Iterable
    - `__iter__`
- Container
    - `__contains__`
- Sized
    - `__len__`
- Callable
- Hashable

<br>

- Iterator
- Sequence
- MutableSequence
- Mapping
- MutableMapping
- Set
- MutableSet
- MappingView
- ItemsView
    - `.items()`
- KeysView
    - `.keys()`
- ValuesView
    - `.values()`

## The Numbers Tower of ABCs
- Number
- Complex
- Real
    - `int`, `bool`, `float`, `fractions.Fraction`
    - Not accept `Decimal`
- Rational
- Integral
    - `int`, `bool`

In [51]:
import numbers
x = 1
y = 1.0
isinstance(x, numbers.Integral), isinstance(y, numbers.Integral)

(True, False)

# Defining and Using and ABC

In [53]:
import abc

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""
    
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        
        This method should raise `LookupError` when the instance is empty.
        """
        
    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [55]:
class Fake(Tombola):
    def pick(self):
        return 13
Fake

__main__.Fake

In [56]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [ ]:
# The preferred declaration of an abstract class
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass

## Subclassing the Tombola ABC
- Python doesn't check if required methods are implemented in subclass of abstract class
    - if necessary methods don't exist, Python throws `RuntimeError`!. This means we can't know missing requirements until run the program.

In [ ]:
import random

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [ ]:
import random

class LotteryBlower(Tombola):
    # flexible because of argument as iterable
    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randomrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

## A Virtual Subclass of Tombola
- An essential characteristic of goose typing is the ability to register a class as a `virtual subclass` of an ABC, even if it does not inherit from it

In [57]:
from random import randrange

@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [58]:
issubclass(TomboList, Tombola)

True

In [59]:
t = TomboList(range(100))
isinstance(t, Tombola)

True